https://huggingface.co/docs/transformers/model_doc/speech_to_text

In [41]:
from subprocess import call


In [54]:
p.get_default_output_device_info()

{'index': 5,
 'structVersion': 2,
 'name': 'Auriculares (WH-1000XM4)',
 'hostApi': 0,
 'maxInputChannels': 0,
 'maxOutputChannels': 2,
 'defaultLowInputLatency': 0.09,
 'defaultLowOutputLatency': 0.09,
 'defaultHighInputLatency': 0.18,
 'defaultHighOutputLatency': 0.18,
 'defaultSampleRate': 44100.0}

In [60]:
for i in range(p.get_device_count()):
  dev = p.get_device_info_by_index(i)
  print((i,dev['name'],dev['maxInputChannels']))

(0, 'Asignador de sonido Microsoft - Input', 2)
(1, 'Microphone (Realtek(R) Audio)', 2)
(2, 'SteelSeries Sonar - Microphone ', 2)
(3, 'Auriculares con micrÃ³fono (WH-1', 1)
(4, 'Asignador de sonido Microsoft - Output', 0)
(5, 'Auriculares (WH-1000XM4)', 0)
(6, 'Altavoces (Realtek(R) Audio)', 0)
(7, 'SteelSeries Sonar - Chat (Steel', 0)
(8, 'SteelSeries Sonar - Aux (SteelS', 0)
(9, 'SteelSeries Sonar - Gaming (Ste', 0)
(10, 'Altavoces (THX Spatial)', 0)
(11, 'ASUS VN247 (NVIDIA High Definit', 0)
(12, 'SteelSeries Sonar - Media (Stee', 0)
(13, 'SteelSeries Sonar - Microphone ', 0)
(14, 'DELL U2421HE (Sonido Intel(R) p', 0)
(15, 'Controlador primario de captura de sonido', 2)
(16, 'Microphone (Realtek(R) Audio)', 2)
(17, 'SteelSeries Sonar - Microphone (SteelSeries Sonar Virtual Audio Device)', 2)
(18, 'Auriculares con micrÃ³fono (WH-1000XM4)', 1)
(19, 'Controlador primario de sonido', 0)
(20, 'Auriculares (WH-1000XM4)', 0)
(21, 'SteelSeries Sonar - Chat (SteelSeries Sonar Virtual Audio D

In [ ]:
import pyaudio
import wave

chunk = 1024  # Record in chunks of 1024 samples
sample_format = pyaudio.paInt16  # 16 bits per sample
channels = 1 #2
fs = 16000  # Record at 44100 samples per second
seconds = 6
filename = "output.wav"

p = pyaudio.PyAudio()  # Create an interface to PortAudio

print('Recording')

stream = p.open(format=sample_format,
                channels=channels,
                rate=fs,
                frames_per_buffer=chunk,
                input_device_index=40,
                input=True)

frames = []  # Initialize array to store frames

# Store data in chunks for 3 seconds
for i in range(0, int(fs / chunk * seconds)):
    data = stream.read(chunk)
    frames.append(data)

# Stop and close the stream 
stream.stop_stream()
stream.close()
# Terminate the PortAudio interface
p.terminate()

print('Finished recording')

# Save the recorded data as a WAV file
wf = wave.open(filename, 'wb')
wf.setnchannels(channels)
wf.setsampwidth(p.get_sample_size(sample_format))
wf.setframerate(fs)
wf.writeframes(b''.join(frames))
wf.close()

Recording


In [101]:
from audio2numpy import open_audio
path = "output.wav"
signal, sampling_rate = open_audio(path)

In [90]:
sampling_rate

16000

In [32]:
import torch
from transformers import Speech2TextProcessor, Speech2TextForConditionalGeneration
from datasets import load_dataset

In [33]:
model = Speech2TextForConditionalGeneration.from_pretrained("facebook/s2t-small-librispeech-asr")

Some weights of Speech2TextForConditionalGeneration were not initialized from the model checkpoint at facebook/s2t-small-librispeech-asr and are newly initialized: ['model.decoder.embed_positions.weights', 'model.encoder.embed_positions.weights']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
processor = Speech2TextProcessor.from_pretrained("facebook/s2t-small-librispeech-asr")

In [34]:
#ds = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")

In [102]:
inputs = processor(signal, sampling_rate=sampling_rate, return_tensors="pt")

In [25]:
#inputs = processor(ds[0]["audio"]["array"], sampling_rate=ds[0]["audio"]["sampling_rate"], return_tensors="pt")

In [103]:
generated_ids = model.generate(inputs["input_features"], attention_mask=inputs["attention_mask"])

In [104]:
transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)

In [105]:
transcription

['together the information from the phoenix i b others']

***

In [16]:
ds[0]["audio"]["sampling_rate"]

16000

In [15]:
ds[0]["audio"]["array"]

array([0.00238037, 0.0020752 , 0.00198364, ..., 0.00042725, 0.00057983,
       0.0010376 ])

+ https://pypi.org/project/audio2numpy/
+ https://realpython.com/python-speech-recognition/

Recording
Finished recording


In [5]:
!dir

 El volumen de la unidad D es Kuky
 El n£mero de serie del volumen es: 3A98-E69F

 Directorio de D:\Github\infobot

23/01/2024  10:04 p. m.    <DIR>          .
21/01/2024  06:01 p. m.    <DIR>          ..
21/01/2024  06:38 p. m.    <DIR>          .ipynb_checkpoints
23/01/2024  10:02 p. m.            11.609 boty.ipynb
23/01/2024  10:04 p. m.           880.684 output.wav
21/01/2024  06:02 p. m.                62 README.md
21/01/2024  06:38 p. m.             6.330 req.txt
               4 archivos        898.685 bytes
               3 dirs  198.792.372.224 bytes libres


In [7]:
P = pyaudio.PyAudio()

stream = P.open(rate=fs, format=pyaudio.paInt16, channels=1, output=True)
stream.write(x.tobytes())

stream.close() # this blocks until sound finishes playing

P.terminate()

AttributeError: 'bytes' object has no attribute 'astype'